# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [27]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [55]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'  # or 'gpt-4o-mini' for a smaller model
openai = OpenAI()

API key looks good so far


In [29]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [30]:
ed = Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 'https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/',
 'https://edwarddonner.com/2025/04/21/the-

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [31]:
link_system_prompt = "You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.\n"

link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [32]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [33]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [34]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/
https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/
https://edwarddo

In [35]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [48]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

# huggingface = Website("https://huggingface.co")
# huggingface.links
biofy = Website("https://biofy.tech/pt-br")
biofy.links

['https://blog.biofy.tech',
 'https://biofytech.vagas.solides.com.br/',
 '/pt-br',
 'https://www.youtube.com/watch?v=X5FTdPtd1lM&list=PLbjCmNG7vwcj-SDBDBtt0jKc9hjxiYyYl',
 'https://www.youtube.com/watch?v=cNIvjJg6XuU',
 'https://www.youtube.com/watch?v=w1B6O8T4KGs',
 'https://www.youtube.com/watch?v=Tpmv6lWz9TU',
 'https://www.youtube.com/watch?v=f4kkcH1Pw_4',
 'https://www.youtube.com/watch?v=X5FTdPtd1lM',
 'https://www.youtube.com/watch?v=aYFjzwPUVi8',
 'https://www.youtube.com/watch?v=1zd9Nfc9qtU',
 'https://www.youtube.com/watch?v=9jhlSJpX9_E',
 'https://www.youtube.com/watch?v=kPgWOh7G74c',
 'https://blog.biofy.tech/',
 'https://blog.biofy.tech/biofy-marca-presenca-no-viii-congresso-cientifico-internacional-do-imepac-com-palestra-de-paulo-perez/',
 'https://blog.biofy.tech/paulo-perez-da-biofy-e-um-dos-palestrantes-confirmados-no-viii-congresso-cientifico-internacional-imepac-sobre-inteligencia-artificial/',
 'https://blog.biofy.tech/biofy-combate-a-resistencia-a-antibioticos-com-

In [49]:
get_links("https://www.biofy.tech/pt-br")

{'links': [{'type': 'about page',
   'url': 'https://www.biofy.tech/pt-br/biofy/quem-somos'},
  {'type': 'cases page', 'url': 'https://www.biofy.tech/pt-br/biofy/cases'},
  {'type': 'careers page', 'url': 'https://biofytech.vagas.solides.com.br/'},
  {'type': 'contact page', 'url': 'https://www.biofy.tech/pt-br/contato'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [50]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [51]:
print(get_all_details("https://www.biofy.tech/pt-br"))

Found links: {'links': [{'type': 'about page', 'url': 'https://www.biofy.tech/pt-br/biofy/quem-somos'}, {'type': 'cases page', 'url': 'https://www.biofy.tech/pt-br/biofy/cases'}, {'type': 'contact page', 'url': 'https://www.biofy.tech/pt-br/contato'}, {'type': 'careers page', 'url': 'https://biofytech.vagas.solides.com.br/'}, {'type': 'blog page', 'url': 'https://blog.biofy.tech'}]}
Landing page:
Webpage Title:
Tela inicial
Webpage Contents:
A Biofy
Soluções
Aplicações
Contato
Blog
Trabalhe conosco
Bem-vindo à Biofy Technologies
Inovação Impulsionada pela Tecnologia
A Biofy Technologies está na vanguarda da revolução digital, fornecendo soluções tecnológicas avançadas para transformar e otimizar diversas indústrias. Nosso compromisso com a inovação e a excelência nos permite oferecer serviços personalizados que atendem às necessidades específicas de cada cliente.
Nossas soluções
Life Science & IA
Solução inovadora que combina biotecnologia e inteligência artificial para avançar na área

In [52]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [53]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [43]:
# get_brochure_user_prompt("HuggingFace", "https://huggingface.co")
get_brochure_user_prompt("Biofy", "https://www.biofy.tech/pt-br")

Found links: {'links': [{'type': 'company page', 'url': 'https://www.biofy.tech/pt-br'}, {'type': 'about page', 'url': 'https://www.biofy.tech/pt-br/biofy/quem-somos'}, {'type': 'cases page', 'url': 'https://www.biofy.tech/pt-br/biofy/cases'}, {'type': 'contact page', 'url': 'https://www.biofy.tech/pt-br/contato'}, {'type': 'careers page', 'url': 'https://biofytech.vagas.solides.com.br/'}, {'type': 'blog page', 'url': 'https://blog.biofy.tech'}]}


'You are looking at a company called: Biofy\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nTela inicial\nWebpage Contents:\nA Biofy\nSoluções\nAplicações\nContato\nBlog\nTrabalhe conosco\nBem-vindo à Biofy Technologies\nInovação Impulsionada pela Tecnologia\nA Biofy Technologies está na vanguarda da revolução digital, fornecendo soluções tecnológicas avançadas para transformar e otimizar diversas indústrias. Nosso compromisso com a inovação e a excelência nos permite oferecer serviços personalizados que atendem às necessidades específicas de cada cliente.\nNossas soluções\nLife Science & IA\nSolução inovadora que combina biotecnologia e inteligência artificial para avançar na área de ciências da vida. Entre suas principais frentes, está o desenvolvimento da ManiGenética, uma plataforma que utiliza IA para analisar dados genéticos e oferecer soluções personali

In [54]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [ ]:
# create_brochure("HuggingFace", "https://huggingface.co")
create_brochure("Biofy", "https://www.biofy.tech/pt-br")

Found links: {'links': [{'type': 'about page', 'url': 'https://www.biofy.tech/pt-br/biofy/quem-somos'}, {'type': 'careers page', 'url': 'https://biofytech.vagas.solides.com.br/'}, {'type': 'blog page', 'url': 'https://blog.biofy.tech/'}]}


# Biofy Technologies Brochure

## Welcome to Biofy Technologies
At Biofy Technologies, we are at the forefront of the digital revolution, delivering advanced technological solutions to transform and optimize various industries. Our commitment to innovation and excellence enables us to provide customized services tailored to the specific needs of each client.

---

## Our Solutions

**Life Sciences & AI**  
Our innovative solutions combine biotechnology with artificial intelligence (AI) to drive advancements in life sciences. Key features include the ManiGenética platform, which utilizes AI to analyze genetic data and provide personalized solutions, along with DNA analysis services. These insights have applications across medicine and agriculture, allowing faster and more precise discoveries in health and biological sciences.

**Data Science & AI**  
We integrate artificial intelligence with data science to revolutionize how businesses use and interpret their data. Our offerings include Generative AI, which creates new content and models from existing data, and Machine Learning for creating efficient predictive models. Our advanced analytics provide deep, actionable insights, empowering organizations to make informed strategic decisions.

**IoT Solutions**  
Our specialized IoT solutions transform connectivity into tangible value for businesses. We develop custom projects that integrate devices and systems to create smart networks. Our expertise extends to creating specific hardware and optimized firmware to ensure seamless and secure operation, connecting the physical world to the digital in innovative ways.

**Software Factory**  
For businesses seeking personalized and optimized enterprise management systems (ERP), our solutions develop custom ERPs powered by AI. These systems automate processes, enhance efficiency, and enable informed decision-making with predictive insights and smart recommendations, elevating performance and competitiveness.

---

## What Our Clients Say

*"Having Biofy as a partner has been a transformative experience for us. The team developed a highly efficient AI software that revolutionized how we manage our internal processes. Customized automation, coupled with intelligent data analysis, has allowed us to optimize our lawyers' time, improve document accuracy, and provide quicker, more effective support to our clients."*  
— Diogo Hemmer, Partner at Hemmer Advocacia

---

## Company Culture

At Biofy, innovation is not just a goal; it is part of our culture. We pride ourselves on a collaborative and inclusive working environment where every team member's ideas are valued. Continuous learning is encouraged, allowing our employees to grow alongside the fast-evolving tech landscape. We believe that fostering creativity and open communication leads to groundbreaking solutions.

---

## Join Us!

We are always looking for passionate individuals to join our team. If you're excited about technology and innovation, check our careers page to find out more about job opportunities at Biofy Technologies!

[Explore Career Opportunities](#)

---

## Stay Connected

Want to learn more? Check out our **Bio Talks** series, where we dive into the fascinating world of technology with industry experts. Join us as we tackle challenging topics like AI's impact beyond ChatGPT, advancements in biotechnology, and the future of IoT in event management.

[Listen to Bio Talks](#)

---

### Contact Us
For inquiries, collaborations, or more information about our services, feel free to reach out to us through our contact page.

--- 

Thank you for considering Biofy Technologies—where innovation meets excellence!

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [44]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [45]:
# stream_brochure("HuggingFace", "https://huggingface.co")
stream_brochure("Biofy", "https://www.biofy.tech/pt-br")

Found links: {'links': [{'type': 'company page', 'url': 'https://www.biofy.tech/pt-br'}, {'type': 'about page', 'url': 'https://www.biofy.tech/pt-br/biofy/quem-somos'}, {'type': 'cases page', 'url': 'https://www.biofy.tech/pt-br/biofy/cases'}, {'type': 'careers page', 'url': 'https://biofytech.vagas.solides.com.br/'}, {'type': 'contact page', 'url': 'https://www.biofy.tech/pt-br/contato'}]}


# Biofy Technologies

Bem-vindo à Biofy Technologies, uma empresa dedicada a impulsionar a inovação tecnológica e a transformação digital em diversas indústrias. Nosso compromisso com a excelência, aliado ao uso das mais modernas tecnologias, nos torna o parceiro ideal para clientes, investidores e futuros colaboradores que buscam soluções completas em Inteligência Artificial, IoT, Fábrica de Software e muito mais.

---

## Quem Somos

A Biofy está na vanguarda da revolução digital, oferecendo soluções avançadas que otimizam processos e tornam organizações mais eficientes. Contamos com equipes altamente especializadas em IA, Data Science, Biotecnologia e desenvolvimento de hardware/firmware para IoT, sempre comprometidas em gerar valor tangível para nossos parceiros.

---

## Nossas Soluções

1. **Life Science & IA**  
   - ManiGenética: plataforma que utiliza IA para análise de dados genéticos e criação de soluções personalizadas.  
   - Análise de DNA: insights detalhados aplicáveis a medicina, agricultura e outras áreas de ciências da vida.  
   - IA aplicada à Biotecnologia: descoberta e inovação em saúde, promovendo avanços rápidos e precisos.

2. **Data Science & IA**  
   - Generative AI: criação de novos conteúdos e modelos a partir de dados existentes.  
   - Data Transformation: preparação e otimização de dados para análises avançadas.  
   - Machine Learning & Advanced Analytics: definição de modelos preditivos e insights profundos para tomadas de decisão estratégicas.

3. **IoT**  
   - Conectividade e integração entre dispositivos e sistemas para redes inteligentes.  
   - Criação de hardware específico para necessidades operacionais e de desempenho.  
   - Desenvolvimento de firmware otimizado para eficiência e segurança, integrando o mundo físico ao digital.

4. **Fábrica de Software**  
   - Desenvolvimento de ERPs personalizados, integrando IA para automação de processos e geração de insights preditivos.  
   - Sistemas sob medida para otimização de rotinas de negócio e tomada de decisão.  
   - Elevação do desempenho e competitividade empresarial por meio de soluções tecnológicas exclusivas.

---

## O Que Nossos Clientes Dizem

> “Ter como parceiro a Biofy é uma experiência transformadora para nós. A equipe desenvolveu um software de inteligência artificial altamente eficiente, que revolucionou a maneira como gerimos nossos processos internos. A automação personalizada, aliada a uma análise de dados inteligente, nos permitiu otimizar o tempo dos nossos advogados, melhorar a precisão na elaboração de documentos jurídicos e oferecer um suporte mais ágil e assertivo aos nossos clientes.”  
> **— Diogo Hemmer, Sócio da Hemmer Advocacia**

---

## Cultura e Bio Talks

Na Biofy, acreditamos em uma cultura de **inovação contínua**, colaborando ativamente para a construção de um futuro mais inteligente e sustentável. Promovemos discussões inspiradoras e compartilhamos conhecimentos por meio do nosso programa **Bio Talks**, onde debatemos temas como:

- “IA Muito Além do ChatGPT” e o impacto da inteligência artificial em nossas vidas.  
- Os desafios e avanços da Biotecnologia.  
- Internet das Coisas (IoT) aplicada ao futuro do setor de eventos.

Para conferir os episódios completos, visite nossa página do **Bio Talks**.

---

## Trabalhe Conosco

Junte-se a um time apaixonado por tecnologia e inovação! A Biofy está sempre em busca de profissionais talentosos e comprometidos em fazer a diferença:

- **Ambiente de colaboração:** Valorizamos o trabalho em equipe e a troca de ideias para promover soluções disruptivas.  
- **Tecnologia de ponta:** Trabalhe com as mais recentes ferramentas e metodologias.  
- **Crescimento profissional:** Oferecemos oportunidades para desenvolvimento de carreira, treinamentos e formação contínua.

Se você deseja fazer parte dessa revolução tecnológica, acesse [Trabalhe Conosco](#) para saber mais sobre vagas e oportunidades.

---

## Entre em Contato

Quer saber como a Biofy pode contribuir para o sucesso do seu negócio ou projeto? Fale conosco e descubra como podemos impulsionar seu crescimento:

- **Telefone:** +55 (XX) XXXX-XXXX  
- **E-mail:** contato@biofy.com.br  
- **Blog:** Explore mais conteúdos e tendências de tecnologia em nosso [Blog](#).

**Biofy Technologies** – Inovação Impulsionada pela Tecnologia.  
Construindo hoje as soluções que moldarão o futuro.

In [ ]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>